<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#original-hotgym-post-training-loop" data-toc-modified-id="original-hotgym-post-training-loop-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>original hotgym post-training loop</a></span></li></ul></div>

trying out Johan Wider's conversion of NuPIC Predictor to community core edition.
https://discourse.numenta.org/t/predictor-learn-before-infer-errors-on-hotgym/7767/2

In [ ]:
from htm.bindings.sdr import SDR, Metrics
from htm.encoders.rdse import RDSE, RDSE_Parameters
from htm.encoders.date import DateEncoder
from htm.bindings.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood #FIXME use TM.anomaly instead, but it gives worse results than the py.AnomalyLikelihood now
from htm.bindings.algorithms import Predictor

In [1]:
from htm.bindings.algorithms import Predictor
from htm.bindings.sdr import SDR, Metrics

import numpy as np

In [ ]:
# the hotgym predictor looks like:
predictor = Predictor(steps=[1,5], alpha=parameters['predictor']['sdrc_alpha'])
predictor_resolution = 1

In [3]:
# Try varying the number of steps and alpha
num_of_steps = 1
alpha = 0.01

predictor = Predictor( steps=[num_of_steps], alpha=alpha )
#classifier = SDRClassifierFactory.create(steps=[num_of_steps], alpha=alpha)

# Make a sequence of SDRs
sequence = []
sdr = SDR((2048, 1))
sdr.sparse = [602, 1449, 1810, 2009]
sequence.append(sdr)
sdr = SDR((2048, 1))
sdr.sparse = [377, 570, 901, 2005]
sequence.append(sdr)
sdr = SDR((2048, 1))
sdr.sparse = [87, 232, 516, 1270]
sequence.append(sdr)

labels = [1, 5, 9]      # Category labels for the sequence

print(sequence[0])
print(sequence[1])
print(sequence[2])

type(sequence)

sequence[0].sparse # sparse returns the non-0 values in an SDR.
# there's probably some way to return their indices and zip the lists


# TRAINING LOOP
# all they do is call predictor.learn()
# in hotgym training loop, run each datapiece through encoders,
# then SP.compute(), then TM.compute()
# and we use predictor.learn(). but they call predictor.infer() before that
# why?
N = 1000
predictor.reset()
for i in range(3*N):
    predictor.learn( i, sequence[i%3], labels[i%3])
#     print(i, type(i))
#     print(sequence[i%3], type(sequence[i%3]))
#     print(labels[i%3], type(labels[i%3]))
#     print()

# - so it's feeding Predictor.learn() an int, an SDR, and another int
#     - it crashes when we feed it a float. so i'm thinking of this wrong, it's a classifier not a float-continuous output predictor... maybe

# so the training loop, using predictor, is something like:
# - for each data+label:
#     - predictor.learn(record_iterator_number, data, label)
#     - predictor.infer() is used to present a prediction after training.

# Give the predictor partial information, and make predictions
predictor.reset()
a = predictor.infer( sequence[0] )
print("Prediction for sequence 0")
print( a[1] )
print(np.argmax( a[1] ) )

a = predictor.infer( sequence[1] )
print("Prediction for sequence 1")
print( a[1] )
print(np.argmax( a[1] ) )

a = predictor.infer( sequence[2] )
print("Prediction for sequence 2")
print( a[1] )
print(np.argmax( a[1] ) )

SDR( 2048, 1 ) 602, 1449, 1810, 2009
SDR( 2048, 1 ) 377, 570, 901, 2005
SDR( 2048, 1 ) 87, 232, 516, 1270
Prediction for sequence 0
[0.00278874038341076, 0.00278874038341076, 0.00278874038341076, 0.00278874038341076, 0.00278874038341076, 0.9748598972891958, 0.002799143516238742, 0.002799143516238742, 0.002799143516238742, 0.002799143516238742]
5
Prediction for sequence 1
[0.0027931382581211184, 0.0027931382581211184, 0.0027931382581211184, 0.0027931382581211184, 0.0027931382581211184, 0.0027931382581211184, 0.0027931382581211184, 0.0027931382581211184, 0.0027931382581211184, 0.9748617099448095]
9
Prediction for sequence 2
[0.0027961843766260383, 0.9748339543943719, 0.0027961843766260383, 0.0027961843766260383, 0.0027961843766260383, 0.0027961843766260383, 0.0027961843766260383, 0.0027961843766260383, 0.0027961843766260383, 0.0027961843766260383]
1


In [25]:
type(a)

dict

In [26]:
a

{1: [0.0006479261565283467,
  0.9941687482689527,
  0.0006479261565283467,
  0.0006479261565283467,
  0.0006479261565283467,
  0.0006479261565283467,
  0.0006479261565283467,
  0.0006479261565283467,
  0.0006479261565283467,
  0.0006479261565283467]}

# original hotgym post-training loop

In [ ]:
  for count, record in enumerate(records):

    # Convert date string into Python date object.
    dateString = datetime.datetime.strptime(record[0], "%m/%d/%y %H:%M")
    # Convert data value string into float.
    consumption = float(record[1])
    inputs.append( consumption )

    # Call the encoders to create bit representations for each value.  These are SDR objects.
    dateBits        = dateEncoder.encode(dateString)
    consumptionBits = scalarEncoder.encode(consumption)

    # Concatenate all these encodings into one large encoding for Spatial Pooling.
    encoding = SDR( encodingWidth ).concatenate([consumptionBits, dateBits])
    enc_info.addData( encoding )

    # Create an SDR to represent active columns, This will be populated by the
    # compute method below. It must have the same dimensions as the Spatial Pooler.
    activeColumns = SDR( sp.getColumnDimensions() )

    # Execute Spatial Pooling algorithm over input space.
    sp.compute(encoding, True, activeColumns)
    sp_info.addData( activeColumns )

    # Execute Temporal Memory algorithm over active mini-columns.
    tm.compute(activeColumns, learn=True)
    tm_info.addData( tm.getActiveCells().flatten() )

    # Predict what will happen, and then train the predictor based on what just happened.
    pdf = predictor.infer( tm.getActiveCells() )
    for n in (1, 5):
      if pdf[n]:
        predictions[n].append( np.argmax( pdf[n] ) * predictor_resolution )
      else:
        predictions[n].append(float('nan'))

    anomalyLikelihood = anomaly_history.anomalyProbability( consumption, tm.anomaly )
    anomaly.append( tm.anomaly )
    anomalyProb.append( anomalyLikelihood )

    predictor.learn(count, tm.getActiveCells(), int(consumption / predictor_resolution))
# Print information & statistics about the state of the HTM.
print("Encoded Input", enc_info)
print("")
print("Spatial Pooler Mini-Columns", sp_info)
print(str(sp))
print("")
print("Temporal Memory Cells", tm_info)
print(str(tm))
print(""
# Shift the predictions so that they are aligned with the input they predict.
for n_steps, pred_list in predictions.items():
  for x in range(n_steps):
      pred_list.insert(0, float('nan'))
      pred_list.pop(
# Calculate the predictive accuracy, Root-Mean-Squared
accuracy         = {1: 0, 5: 0}
accuracy_samples = {1: 0, 5: 0
for idx, inp in enumerate(inputs):
  for n in predictions: # For each [N]umber of time steps ahead which was predicted.
    val = predictions[n][ idx ]
    if not math.isnan(val):
      accuracy[n] += (inp - val) ** 2
      accuracy_samples[n] += 1
for n in sorted(predictions):
  accuracy[n] = (accuracy[n] / accuracy_samples[n]) ** .5
  print("Predictive Error (RMS)", n, "steps ahead:", accuracy[n]
# Show info about the anomaly (mean & std)
print("Anomaly Mean", np.mean(anomaly))
print("Anomaly Std ", np.std(anomaly)
# Plot the Predictions and Anomalies.
if verbose:
  try:
    import matplotlib.pyplot as plt
  except:
    print("WARNING: failed to import matplotlib, plots cannot be shown.")
    return -accuracy[5
  plt.subplot(2,1,1)
  plt.title("Predictions")
  plt.xlabel("Time")
  plt.ylabel("Power Consumption")
  plt.plot(np.arange(len(inputs)), inputs, 'red',
           np.arange(len(inputs)), predictions[1], 'blue',
           np.arange(len(inputs)), predictions[5], 'green',)
  plt.legend(labels=('Input', '1 Step Prediction, Shifted 1 step', '5 Step Prediction, Shifted 5 steps')
  plt.subplot(2,1,2)
  plt.title("Anomaly Score")
  plt.xlabel("Time")
  plt.ylabel("Power Consumption")
  inputs = np.array(inputs) / max(inputs)
  plt.plot(np.arange(len(inputs)), inputs, 'red',
           np.arange(len(inputs)), anomaly, 'blue',)
  plt.legend(labels=('Input', 'Anomaly Score'))
  plt.show(
return -accuracy[5]